```
@beelzebruno
2022
```

<hr />

# Save te princess

This is an ludic abstraction to a graph extraction from a given matrix.
Sometimes we got problems that implies the use os search algorithms to find a
solution to a problem, and this problem comes presented in the shape of a matrix.

Suppose we have a scenary where theres a lost princess somewhere in a region
mapped in a (x, y) coordinates chart, as well we have a hero in the mission of
find and rescue the lost princess:

![lost princess map](https://github.com/brunolcarli/GooGraph/blob/master/static/img/hero_princess_map.png?raw=true)

The red **X** marks are places the hero cannot pass or access.

**Find solutions to help our hero search the best path to the lost princess**

In [77]:
from random import choice
import pandas as pd
import matplotlib.pyplot as plt
import seaborn

plt.style.use('seaborn')

In [11]:

numbers = list(range(70))
matrix = [
    numbers[:10],
    numbers[10:20],
    numbers[20:30],
    numbers[30:40],
    numbers[40:50],
    numbers[50:60],
    numbers[60:70]
]
walls = [
    (3, 0), (1, 1), (2, 3), (3, 3), (4, 3),
    (4, 4), (6, 4), (1, 5), (4, 5), (1, 7),
    (5, 7), (3, 8), (6, 8), (2, 9)
]
for wall in walls:
    x, y = wall
    matrix[x][y] = None

matrix_df = pd.DataFrame(matrix)
matrix_df

,0,1,2,3,4,5,6,7,8,9
0,0.0,1.0,2,3.0,4.0,5.0,6,7.0,8.0,9.0
1,10.0,NaN,12,13.0,14.0,NaN,16,NaN,18.0,19.0
2,20.0,21.0,22,NaN,24.0,25.0,26,27.0,28.0,NaN
3,NaN,31.0,32,NaN,34.0,35.0,36,37.0,NaN,39.0
4,40.0,41.0,42,NaN,NaN,NaN,46,47.0,48.0,49.0
5,50.0,51.0,52,53.0,54.0,55.0,56,NaN,58.0,59.0
6,60.0,61.0,62,63.0,NaN,65.0,66,67.0,NaN,69.0


In [144]:
class Node:
    def __init__(self, value, coord):
        self.value = value
        self.coord = coord
        self.connections = []

    def __repr__(self):
        return str(self.value)


class Graph:
    def __init__(self, nodes=None):
        self.nodes = nodes or {}


class Agent:
    def __init__(self, world, start_position: tuple, target: tuple):
        self.world = world
        self.current_position = start_position
        self.previous_position = None
        self.movements = 0
        self.target = target
        self.graph = Graph()
        self.buffer = [self.current_position]

    def possible_moves(self, reference):
        x, y = reference
        return (
            (x - 1, y),  # up
            (x + 1, y),  # down
            (x, y + 1),  # right
            (x, y - 1)  # left
        )

    @property
    def node(self):
        """ Return current position as Node object """
        x, y = self.current_position
        return Node(self.world[x][y], self.current_position)

    def to_node(self, position):
        """ Return a position as Node object """
        x, y = position
        return Node(self.world[x][y], position)

    def valid_movement(self, position: tuple):
        """ Check if position to move is a valid position """
        new_x, new_y = position

        invalid_x_range = new_x > len(self.world)-1 or new_x < 0
        invalid_y_range = new_y > len(self.world[0])-1 or new_y < 0
        if invalid_x_range or invalid_y_range:
            return False

        new_position = self.world[new_x][new_y]
        if new_position is None:
            # is a wall, cannot pass
            return False
        
        # check if movement reachable
        x, y = self.current_position
        if abs(new_x - x) > 1 or abs(new_y - y) > 1:
            # maxx step size is one
            return False

        # is valid to move
        return True

    def move(self, position: tuple):
        if not self.valid_movement(position):
            raise Exception(f'Cannot move to the position {position}')

        self.graph.nodes[str(self.node)] = self.node
        previous_node = str(self.node)
        self.previous_position = self.current_position
        self.current_position = position
        self.graph.nodes[previous_node].connections.append(self.node)
        self.buffer.append(self.current_position)
        self.movements += 1
        return self

    def check_position(self):
        """
        Return True if princess is in the current position.
        """
        return self.current_position == self.target

    def select_best_option(self):
        # valid options given the current position
        options = [move for move in self.possible_moves(self.current_position)
                   if move not in self.buffer and self.valid_movement(move)]
        if not options:
            return
        print(options)
        possibilities = []
        for option in options:
            possibilities.append(self.a_star(
                option,
                self.target
            ))

        return sorted(possibilities, key=lambda x: x[1])[0][0]

    def bfs(self, start, target):
        """ Breadth first search """
        queue = [start]
        moves = 0
        while queue:
            position = queue.pop(0)
            moves += 1
            if position == target:
                return position, moves

            for area in self.possible_moves(position):
                queue.append(area)
        
        return position, moves

    def a_star(self, start, target):
        """ Breadth first search """
        h = lambda x: self.to_node(x).value - self.to_node(target).value
        f = lambda x: len(self.buffer) + h(x)
        explored = set()
        queue = [(start, f(start))]

        while queue:
            position, _ = queue.pop(0)

            if position == target:
                return position, f(position)

            explored.add(position)
            areas = [area for area in self.possible_moves(position)
                     if self.valid_movement(area)]
            for area in areas:
                if area not in explored:
                    queue.append((area, f(area)))
        
            queue = sorted(queue, key=lambda k: k[1])
        return position, f(position)

    def run(self):
        while not self.check_position():
            next_move = self.select_best_option()
            if not next_move:
                break

            self.move(next_move)
        print('Done')






In [134]:
len(matrix[0])

10

In [143]:
PRINCESS_POSITION = (1, 8)
graph = Graph()

hero = Agent(matrix, (5, 1), PRINCESS_POSITION)
# hero.a_star(hero.current_position, PRINCESS_POSITION)
hero.run()


[((6, 2), 45), ((6, 2), 45), ((6, 2), 45), ((6, 2), 45)]
[((6, 3), 47), ((6, 2), 46), ((6, 3), 47)]
[((6, 3), 48), ((6, 2), 47), ((6, 3), 48)]
[((6, 3), 49), ((6, 2), 48), ((6, 3), 49)]
[((6, 3), 50), ((6, 2), 49), ((6, 3), 50)]
[((6, 3), 51), ((6, 2), 50), ((6, 3), 51)]
[((6, 3), 52), ((6, 2), 51), ((6, 3), 52)]
[((6, 3), 53), ((6, 2), 52), ((6, 3), 53)]
[((6, 3), 54), ((6, 2), 53), ((6, 3), 54)]
[((6, 3), 55), ((6, 2), 54), ((6, 3), 55)]
[((6, 3), 56), ((6, 2), 55), ((6, 3), 56)]
[((6, 3), 57), ((6, 2), 56), ((6, 3), 57)]
[((6, 3), 58), ((6, 2), 57), ((6, 3), 58)]
[((6, 3), 59), ((6, 2), 58), ((6, 3), 59)]
[((6, 3), 60), ((6, 2), 59), ((6, 3), 60)]
[((6, 3), 61), ((6, 2), 60), ((6, 3), 61)]
[((6, 3), 62), ((6, 2), 61), ((6, 3), 62)]
[((6, 3), 63), ((6, 2), 62), ((6, 3), 63)]
[((6, 3), 64), ((6, 2), 63), ((6, 3), 64)]
[((6, 3), 65), ((6, 2), 64), ((6, 3), 65)]
[((6, 3), 66), ((6, 2), 65), ((6, 3), 66)]
[((6, 3), 67), ((6, 2), 66), ((6, 3), 67)]
[((6, 3), 68), ((6, 2), 67), ((6, 3), 68

KeyboardInterrupt: 

In [140]:
hero.movements

11189

In [141]:
hero.buffer

[(5, 1),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 (6, 2),
 